In [12]:
from loaders.cognicity_loader import CognicityLoader
# from image_recognition.aws_recog import AwsLabeler
# from image_recognition.goog_recog import GoogleLabeler
#import chennai_config
#config = chennai_config.config

import jakarta_config
config = jakarta_config.config
from learners.svm_learner import SvmLearner

# from nlp.bow_labeler import BowLabeler
# learn = SvmLearner(config, CognicityLoader, BowLabeler)


from image_recognition.goog_recog import GoogleLabeler
learn = SvmLearner(config, CognicityLoader, GoogleLabeler)

# from image_recognition.aws_recog import AwsLabeler
# learn = SvmLearner(config, CognicityLoader, AwsLabeler)

import random
validation_set = set(random.sample(config["flood_pkeys"], 5))
validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 5)))
params = {"T":1000}
learn.train( params, validation_set)

2019-08-13 10:50:21,078 - DEBUG - CognicityLoader constructed
2019-08-13 10:50:21,080 - DEBUG - AwsLabeler constructed
2019-08-13 10:50:21,081 - DEBUG - CognicityLoader constructed
2019-08-13 10:50:21,083 - DEBUG - AwsLabeler constructed
2019-08-13 10:50:21,084 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-13 10:50:37,487 - INFO - Num Correct 7 Out of 10
2019-08-13 10:50:37,488 - INFO - Val score: 0.7


(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , 24.70741081,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [22.2561245 , 20.75111961,  0.        , ...,  0.        ,
         28.47358704,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.]]))

In [13]:
val = learn.val_data_w_pkey[1:,:].T
learn.clf.predict(learn.val_data_w_pkey[1:,:].T)

array([-1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.])

In [17]:
dat = learn.t_data_w_pkey[1:,:].T
lab = learn.t_labels[0, :]
# learn.clf.decision_function(dat)

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm


param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

cv = GridSearchCV(svm.SVC(), param_grid, cv=10, n_jobs=5)

cv.fit(dat, lab)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=5,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [27]:
sorted(cv.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_C',
 'param_gamma',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'split5_test_score',
 'split6_test_score',
 'split7_test_score',
 'split8_test_score',
 'split9_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [28]:
cv.cv_results_

{'mean_fit_time': array([16.32301688, 17.03301001, 14.97846949, 16.05208921, 20.93809068,
        21.76419857, 20.68259163, 22.29693329, 21.8470614 , 21.72499647,
        20.55633793, 20.06107321]),
 'mean_score_time': array([0.97471123, 1.08129294, 1.02297051, 1.13616841, 2.21788049,
        2.4151747 , 2.39297657, 2.36481414, 2.36193008, 2.32443864,
        2.18048482, 2.06993895]),
 'mean_test_score': array([0.60342497, 0.60342497, 0.60342497, 0.60342497, 0.52996845,
        0.53086976, 0.52996845, 0.53132041, 0.52996845, 0.53132041,
        0.52996845, 0.53132041]),
 'param_C': masked_array(data=[1, 10, 100, 1000, 1, 1, 10, 10, 100, 100, 1000, 1000],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=[--, --, --, --, 0.001, 0.0001, 0.001, 0.0001, 0.001,
                    0.0001, 0.001, 0.0001],
              mask=[ True,  

In [25]:
cv.best_estimator_
# for jakarta aws data:
# SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
#     kernel='linear', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)

# cv.best_score_ = 0.5971158179360072

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [29]:
cv.best_score_

0.6034249662009914